In [1]:
import numpy as np
import pandas as pd
import  matplotlib.pyplot as plt
from  sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest,chi2,RFE
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from imblearn.over_sampling import SMOTE,ADASYN

from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import  make_pipeline,Pipeline
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import f_regression
from sklearn.metrics import confusion_matrix,classification_report



In [2]:
url="https://raw.githubusercontent.com/digipodium/Datasets/main/regression/ames_housing_no_missing.csv"
df=pd.read_csv(url,index_col=False)
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0,Gd,MnPrv,Shed,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Grvl,Reg,Lvl,AllPub,FR2,...,0,Gd,MnPrv,Shed,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,Grvl,IR1,Lvl,AllPub,Inside,...,0,Gd,MnPrv,Shed,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,Grvl,IR1,Lvl,AllPub,Corner,...,0,Gd,MnPrv,Shed,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,Grvl,IR1,Lvl,AllPub,FR2,...,0,Gd,MnPrv,Shed,0,12,2008,WD,Normal,250000


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1460 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          1460 non-null   object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [3]:
X=df.drop(columns="SalePrice")
y=df["SalePrice"]

In [4]:
numeric_transformer = Pipeline(
            steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
        )

categorical_transformer = OneHotEncoder(handle_unknown="ignore")
preprocessor = ColumnTransformer(
            transformers=[
                ("numeric", numeric_transformer, X.select_dtypes(np.number).columns.tolist()),
                ("category", categorical_transformer,X.select_dtypes("object").columns.tolist()),
            ]
        )


In [5]:
xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=45)

In [6]:

feature_selector = Pipeline(
        steps=[("preprocessor", preprocessor),
        ("feature", SelectKBest(f_regression,k=3))]
    )
feature_selector.fit(xtrain,ytrain)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'Low...
                                                   'Neighborhood', 'Condition1',
                                                   'Condition2', 'BldgType',
                                                   'HouseStyle', 'RoofStyle',
                                                   'RoofMatl', 'Exterior1st',
                                                   'Exterior2nd', 'MasVnrType',
                                                   'ExterQual', 'ExterCond',
                                                   'Foundation', 'BsmtQual',
                                                   'BsmtCond', 'BsmtExposure',
                                                   'BsmtFinType1',
                                                   'BsmtFinType2', 'Heating',
                                                   'HeatingQC', 'CentralAir',
                                                   'Electrical', ...])])),
                ('feature',
                 SelectKBest(k=3,
                             score_func=<function f_regression at 0x0000020CCA3D3CA0>))])

In [17]:
X = df[['OverallQual', 'GrLivArea', 'GarageCars']]
xtrain,xtest,ytrain,ytest=train_test_split(X.values,y.values,test_size=0.2,random_state=45)

In [20]:
model_selector = Pipeline(
        steps=[("scaling", StandardScaler()),
        ("classifier", RandomForestRegressor())]
    )
model_selector.fit(xtrain,ytrain)
model_selector.fit(xtrain,ytrain)
ypred=model_selector.predict(xtest)
result=r2_score(ytest,ypred)
print(f"R2score is {result}")

R2score is 0.7990233635454504


In [ ]:
# model_selector = Pipeline(
#         steps=[("preprocessor", preprocessor),
#         ("feature", SelectKBest(f_regression,k=3)),
#         ("classifier", RandomForestRegressor())]
#     )
# model_selector.fit(xtrain,ytrain)
# model_selector.fit(xtrain,ytrain)
# ypred=model_selector.predict(xtest)
# result=r2_score(ytest,ypred)
# print(f"R2score is {result}")

In [8]:
print(feature_selector.get_feature_names_out())

['numeric__OverallQual' 'numeric__GrLivArea' 'numeric__GarageCars']


In [17]:
xtrain["GrLivArea"]

1075    1740
372      752
502     1214
924     1686
662     1392
        ... 
580     1429
163      882
607     2008
414     2097
971     1626
Name: GrLivArea, Length: 1168, dtype: int64

In [21]:
inp=np.array([7,1740,1])
print(model_selector.predict([inp]))
print(type(model_selector.predict([inp])))
    

[204609.]
<class 'numpy.ndarray'>


In [22]:
model_selector

Pipeline(steps=[('scaling', StandardScaler()),
                ('classifier', RandomForestRegressor())])